# Ch 15. BERT 사전학습모형에 대한 미세조정학습

## 15.1 BERT 학습을 위한 전처리



In [1]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

sentence1 = "What a beautiful day!"
sentence2 = "Nvidia Titan XP has 12GB of VRAM"

# 1. 토큰화 결과
print(sentence1, '토큰화 결과:', tokenizer.tokenize(sentence1))
print(sentence2, '토큰화 결과:', tokenizer.tokenize(sentence2))

What a beautiful day! 토큰화 결과: ['what', 'a', 'beautiful', 'day', '!']
Nvidia Titan XP has 12GB of VRAM 토큰화 결과: ['n', '##vid', '##ia', 'titan', 'xp', 'has', '12', '##gb', 'of', 'vr', '##am']


In [2]:
# 2. BERT 모형 입력 생성
inputs = tokenizer([sentence1, sentence2], padding=True)
print('BERT 입력:', inputs)

BERT 입력: {'input_ids': [[101, 2054, 1037, 3376, 2154, 999, 102, 0, 0, 0, 0, 0, 0], [101, 1050, 17258, 2401, 16537, 26726, 2038, 2260, 18259, 1997, 27830, 3286, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [3]:
# 3. 두 문장으로 이루어진 시퀀스에 대한 BERT 모형 입력 생성
inputs = tokenizer(sentence1, sentence2, padding=True)
print('두 문장 시퀀스에 대한 BERT 입력:', inputs)

두 문장 시퀀스에 대한 BERT 입력: {'input_ids': [101, 2054, 1037, 3376, 2154, 999, 102, 1050, 17258, 2401, 16537, 26726, 2038, 2260, 18259, 1997, 27830, 3286, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


## 15.2 트랜스포머의 트레이너를 이용한 미세조정학습


In [4]:
import nltk
from nltk.corpus import movie_reviews
from sklearn.model_selection import train_test_split #sklearn에서 제공하는 split 함수를 사용
import numpy as np

nltk.download('movie_reviews')
fileids = movie_reviews.fileids() #movie review data에서 file id를 가져옴
reviews = [movie_reviews.raw(fileid) for fileid in fileids] #file id를 이용해 raw text file을 가져옴
categories = [movie_reviews.categories(fileid)[0] for fileid in fileids] 

# label을 0, 1의 값으로 변환
label_dict = {'pos':1, 'neg':0}
y = [label_dict[c] for c in categories]

X_train, X_test, y_train, y_test = train_test_split(reviews, y, test_size=0.2, random_state=7)

print('Train set count: ', len(X_train))
print('Test set count: ', len(X_test))

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\momdad\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


Train set count:  1600
Test set count:  400


In [5]:
from transformers import BertTokenizerFast, BertForSequenceClassification 

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")

train_input = tokenizer(X_train, truncation=True, padding=True, return_tensors="pt")
test_input = tokenizer(X_test, truncation=True, padding=True, return_tensors="pt")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


(1) https://tutorials.pytorch.kr/beginner/basics/data_tutorial.html

In [6]:
import torch

class OurDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = OurDataset(train_input, y_train)
test_dataset = OurDataset(test_input, y_test)

In [7]:
#!pip install datasets

#https://huggingface.co/docs/datasets/

In [8]:
#from datasets import load_metric
#metric = load_metric("accuracy")
# metric 관련된 라이브러리가 datasets에서 evaluate로 분리되면서 이전 코드는 작동하지 않게 됨
# evalute 패키지 참고: https://huggingface.co/docs/evaluate/a_quick_tour

import evaluate
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [9]:
# accelerate가 설치되지 않으면 아래와 같은 에러가 발생
#Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]`
#!pip install transformers

(1) https://huggingface.co/transformers/main_classes/trainer.html?highlight=trainingarguments#transformers.TrainingArguments

(2) https://huggingface.co/transformers/main_classes/trainer.html?highlight=trainingarguments#transformers.Trainer

In [10]:
from transformers import Trainer, TrainingArguments

# Trainer에서 사용할 하이퍼 파라미터 지정
training_args = TrainingArguments(
    output_dir='./results',          # 모형 예측이나 체크포인트 출력 폴더, 반드시 필요함
    num_train_epochs=2,              # 학습 에포크 수
    per_device_train_batch_size=8,   # 학습에 사용할 배치 사이즈
    per_device_eval_batch_size=16,   # 평가에 사용할 배치 사이즈
)

trainer = Trainer(
    model=model,                     # 학습할 모형
    args=training_args,              # 위에서 정의한 학습 매개변수
    train_dataset=train_dataset,     # 훈련 데이터셋
    compute_metrics=compute_metrics,
)

# 미세조정학습 실행
trainer.train()

C:\Users\momdad\AppData\Local\Temp\ipykernel_7656\296560984.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}
C:\Users\momdad\anaconda3\envs\llm\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
C:\Users\momdad\anaconda3\envs\llm\Lib\site-packages\torch\nn\parallel\_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
C:\

Step,Training Loss


TrainOutput(global_step=100, training_loss=0.4461402893066406, metrics={'train_runtime': 77.8219, 'train_samples_per_second': 41.12, 'train_steps_per_second': 1.285, 'total_flos': 841955377152000.0, 'train_loss': 0.4461402893066406, 'epoch': 2.0})

In [11]:
trainer.evaluate(eval_dataset=test_dataset)

C:\Users\momdad\AppData\Local\Temp\ipykernel_7656\296560984.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}
C:\Users\momdad\anaconda3\envs\llm\Lib\site-packages\torch\nn\parallel\_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.3152330219745636,
 'eval_accuracy': 0.875,
 'eval_runtime': 2.7241,
 'eval_samples_per_second': 146.835,
 'eval_steps_per_second': 2.57,
 'epoch': 2.0}

## 15.3 파이토치를 이용한 미세조정학습



In [12]:
del model
del trainer
torch.cuda.empty_cache()

In [13]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=8)

In [14]:
from transformers import BertModel

bert_model = BertModel.from_pretrained('bert-base-uncased')

In [15]:
# BERT를 포함한 신경망 모형
class MyModel(torch.nn.Module):
    def __init__(self, pretrained_model, token_size, num_labels): 
        super(MyModel, self).__init__()
        self.token_size = token_size
        self.num_labels = num_labels
        self.pretrained_model = pretrained_model

        # 분류기 정의
        self.classifier = torch.nn.Linear(self.token_size, self.num_labels)

    def forward(self, inputs):
        # BERT 모형에 입력을 넣고 출력을 받음
        outputs = self.pretrained_model(**inputs)
        # BERT 출력에서 CLS 토큰에 해당하는 부분만 가져옴
        bert_clf_token = outputs.last_hidden_state[:,0,:]
        
        return self.classifier(bert_clf_token)

# token_size는 BERT 토큰과 동일, bert_model.config.hidden_size로 알 수 있음
model = MyModel(bert_model, num_labels=2, token_size=bert_model.config.hidden_size) 

In [16]:
from transformers import AdamW
import torch.nn.functional as F
import time

# GPU 가속을 사용할 수 있으면 device를 cuda로 설정하고, 아니면 cpu로 설정
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)  # 모형을 GPU로 복사
model.train()     # 학습모드로 전환

optim = AdamW(model.parameters(), lr=5e-5) # 옵티마이저를 트랜스포머가 제공하는 AdamW로 설정
criterion = torch.nn.CrossEntropyLoss()    # 멀티클래스이므로 크로스 엔트로피를 손실함수로 사용

start = time.time() # 시작시간 기록
num_epochs = 4      # 학습 epoch를 4회로 설정
for epoch in range(num_epochs):
    total_epoch_loss = 0  # epoch의 총 loss 초기화
    for step, batch in enumerate(train_loader):
        optim.zero_grad()     # 그래디언트 초기화
        # 배치에서 label을 제외한 입력만 추출하여  GPU로 복사
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'} 
        labels = batch['labels'].to(device) # 배치에서 라벨을 추출하여 GPU로 복사
        outputs = model(inputs) # 모형으로 결과 예측
        # 두 클래스에 대해 예측하고 각각 비교해야 하므로 labels에 대해 원핫인코딩을 적용한 후에 손실을 게산
        loss = criterion(outputs, F.one_hot(labels, num_classes=2).float()) # loss 계산

        if (step+1) % 100 == 0: # 100 배치마다 경과한 시간과 loss를 출력
            elapsed = time.time() - start
            print('Epoch %d, batch %d, elapsed time: %.2f, loss: %.4f' % (epoch+1, step+1, elapsed, loss))
        total_epoch_loss += loss
        loss.backward() # 그래디언트 계산
        optim.step()    # 가중치 업데이트
    avg_epoch_loss = total_epoch_loss / len(train_loader) # epoch의 평균 loss 계산
    print('Average loss for epoch %d: %.4f' % (epoch+1, avg_epoch_loss))


C:\Users\momdad\anaconda3\envs\llm\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
C:\Users\momdad\AppData\Local\Temp\ipykernel_7656\296560984.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}


Epoch 1, batch 100, elapsed time: 29.24, loss: 0.6078
Epoch 1, batch 200, elapsed time: 59.22, loss: 1.0248
Average loss for epoch 1: 0.6265
Epoch 2, batch 100, elapsed time: 89.19, loss: 0.4776
Epoch 2, batch 200, elapsed time: 119.15, loss: 0.0759
Average loss for epoch 2: 0.3587
Epoch 3, batch 100, elapsed time: 149.38, loss: 0.0788
Epoch 3, batch 200, elapsed time: 179.40, loss: 0.2617
Average loss for epoch 3: 0.1959
Epoch 4, batch 100, elapsed time: 209.56, loss: 0.0444
Epoch 4, batch 200, elapsed time: 239.61, loss: 0.0036
Average loss for epoch 4: 0.0834


In [18]:
#from datasets import load_metric
#metric = load_metric("accuracy")
# metric 관련된 라이브러리가 datasets에서 evaluate로 분리되면서 이전 코드는 작동하지 않게 됨
# evalute 패키지 참고: https://huggingface.co/docs/evaluate/a_quick_tour

import evaluate
metric = evaluate.load("accuracy")

test_loader = DataLoader(test_dataset, batch_size=16)

model.eval()
for batch in test_loader:
    inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
    labels = batch['labels'].to(device)
    
    with torch.no_grad(): # 학습할 필요가 없으므로 그래디언트 계산을 끔
        outputs = model(inputs)
        #print(outputs)

    predictions = torch.argmax(outputs, dim=-1)
    metric.add_batch(predictions=predictions, references=labels)

metric.compute()

C:\Users\momdad\AppData\Local\Temp\ipykernel_7656\296560984.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}


{'accuracy': 0.85}